In [1]:
# Loading libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,f1_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score


In [14]:
# Loading dataset

dataset = pd.read_csv('../input/creditcard.csv')
dataset = dataset.drop(['V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)
dataset['V1_'] = dataset.V1.map(lambda x: 1 if x < -3 else 0)
dataset['V2_'] = dataset.V2.map(lambda x: 1 if x > 2.5 else 0)
dataset['V3_'] = dataset.V3.map(lambda x: 1 if x < -4 else 0)
dataset['V4_'] = dataset.V4.map(lambda x: 1 if x > 2.5 else 0)
dataset['V5_'] = dataset.V5.map(lambda x: 1 if x < -4.5 else 0)
dataset['V6_'] = dataset.V6.map(lambda x: 1 if x < -2.5 else 0)
dataset['V7_'] = dataset.V7.map(lambda x: 1 if x < -3 else 0)
dataset['V9_'] = dataset.V9.map(lambda x: 1 if x < -2 else 0)
dataset['V10_'] = dataset.V10.map(lambda x: 1 if x < -2.5 else 0)
dataset['V11_'] = dataset.V11.map(lambda x: 1 if x > 2 else 0)
dataset['V12_'] = dataset.V12.map(lambda x: 1 if x < -2 else 0)
dataset['V14_'] = dataset.V14.map(lambda x: 1 if x < -2.5 else 0)
dataset['V16_'] = dataset.V16.map(lambda x: 1 if x < -2 else 0)
dataset['V17_'] = dataset.V17.map(lambda x: 1 if x < -2 else 0)
dataset['V18_'] = dataset.V18.map(lambda x: 1 if x < -2 else 0)
dataset['V19_'] = dataset.V19.map(lambda x: 1 if x > 1.5 else 0)
dataset['V21_'] = dataset.V21.map(lambda x: 1 if x > 0.6 else 0)

#Crea una nueva función para transacciones normales (no fraudulentas).
dataset.loc[dataset.Class == 0, 'Clase'] = 0
dataset.loc[dataset.Class == 1, 'Clase'] = 1
dataset = dataset.drop(['Class'], axis =1)

In [15]:
# Parameters and results
# Disminuya todas las características que tienen distribuciones muy similares entre los dos tipos de transacciones.
print(dataset.shape[1])
n_len=dataset.shape[1]
pd.set_option("display.max_columns",101)
dataset.head()

x = dataset.iloc[: , 1:n_len-1].values
y = dataset.iloc[:, n_len-1].values

37


In [16]:
# Handling Missing Values
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(x[:, 1::n_len-1])
x[:, 1::n_len-1] = imputer.fit_transform(x[:, 1::n_len-1])

# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x = sc_x.fit_transform(x)

#FOLDS

In [17]:
#from sklearn.model_selection import KFold, cross_validate
#scoring = ['accuracy', 'f1']
#kfold = KFold(n_splits=3)
#kfold = KFold(n_splits=6, random_state=0)

In [18]:
#results = cross_validate(rf_classifier,x,y,cv=kfold, scoring=scoring,return_train_score=False)


#  XgBoost Model

In [19]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y,test_size = 0.25, random_state = 42)

In [20]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
train_X, train_Y = sm.fit_sample(train_x, train_y)

In [21]:
import xgboost as xgb
xgb_classifier=xgb.XGBClassifier(colsample_bytree= 0.8, learning_rate= 0.3,
                                 min_child_weight= 1.0, n_estimators= 100,
                                 subsample= 0.7,reg_lambda= 0.1, max_depth= 15,gamma= 1.0)


In [22]:
xgb_classifier.fit(train_x, train_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=1.0, learning_rate=0.3, max_delta_step=0, max_depth=15,
       min_child_weight=1.0, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=0.1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.7)

In [23]:
y_predict = xgb_classifier.predict(test_x)

In [26]:
from sklearn.metrics import accuracy_score
accura= accuracy_score(test_y, y_predict)  

from sklearn.metrics import f1_score
f1score= f1_score(test_y, y_predict)  

from sklearn.metrics import recall_score
recall= recall_score(test_y, y_predict)  



In [27]:
print("accuracy: ",accura)
print("f1_score: ",f1score)
print("recall: ",recall)

('accuracy: ', 0.99957866352068758)
('f1_score: ', 0.85576923076923084)
('recall: ', 0.78761061946902655)
